In [1]:
import pandas as pd
from tqdm import tqdm
import spacy
import en_core_web_lg

# from spacy.tokenizer import Tokenizer
dataFolder = '/data2/Datasets/Raw'

import operator
import sys
sys.path.insert(0,'/data2/Datasets/')
from preprocess import *

In [2]:
!python -m spacy download en_core_web_lg


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
# GETTING POS TAGS(majority voting) OF WORDS FROM ORIGINAL DATASETS
lexiconDatasetList = ["nrc_joy", "nrc_sadness", "nrc_anger", "nrc_fear", "nrc_surprise", "amazon_toys_subset","amazon_finefood_subset", "yelp_subset", "empathy"]

final_pos_dicts = {}
for lexiconDataset in lexiconDatasetList:
    token_pos_dict = {}
    token_freq_dict = {}
    final_token_pos_dict = {}

    totalDF = getData(dataFolder,lexiconDataset)
    nlp = en_core_web_lg.load()

    with nlp.disable_pipes():
        for msg in tqdm(totalDF["text"]): 
            doc = nlp(msg)
            for tkn in doc:
                if str(tkn) not in token_pos_dict:
                    token_pos_dict[str(tkn)] = {}

                if str(tkn) not in token_freq_dict:
                    token_freq_dict[str(tkn)] = 0
                token_freq_dict[str(tkn)] += 1

                if tkn.pos_ not in token_pos_dict[tkn.text]:
                    token_pos_dict[str(tkn)][tkn.pos_]=0
                token_pos_dict[str(tkn)][tkn.pos_]+=1


    for tkn in token_pos_dict:
        final_token_pos_dict[tkn] = max(token_pos_dict[tkn].items(), key=operator.itemgetter(1))[0]
    final_pos_dicts[lexiconDataset] = final_token_pos_dict

100%|██████████| 1860/1860 [00:40<00:00, 46.34it/s]


In [8]:

# BERT_Mask       DistilBERT_Mask       FFN           LSTM
# BERT_Partition  DistilBERT_Partition  FFN_DeepShap  SVM
# method_path_name ="DistilBERT_Mask/"
# lexica_path = "~/Final/embedding-lexica-creation/lexica/"
# curr_lex = "yelp_subset"
# csv_file_name = "yelp_subset_distilbert_regression_mask.csv"
# text_name = "Word"


def calculatePOSStatistics(method_path_name, lexica_path, curr_lex, text_name):
    lexica =  pd.read_csv(lexica_path+method_path_name+csv_file_name)
    pos_freq_dict = {'PRON':0, 'AUX':0, 'ADV':0, 'VERB':0, 'PART':0, 'ADP':0, 'DET':0, 'NOUN':0, 'PUNCT':0, 'SPACE':0, 'ADJ':0, 'CCONJ':0, 'SCONJ':0, 'PROPN':0, 'NUM':0,'INTJ':0, 'X':0, 'SYM':0,'UNCLSFD':0, 'valid_count':0}
    total_valid_count = 0    
    words_not_found =[]

    for word in tqdm(lexica[text_name]): 
        if word not in final_pos_dicts[curr_lex]:
            pos_freq_dict["UNCLSFD"]+=1
        else:
            pos_freq_dict[final_pos_dicts[curr_lex][word]]+=1
            total_valid_count+=1
    return pos_freq_dict


In [11]:
# CHANGE path name, lexica path, curr lex, csv filename and text name here and RUN

method_path_name ="DistilBERT_Mask/"
lexica_path = "~/Final/embedding-lexica-creation/lexica/"
curr_lex = "yelp_subset"
csv_file_name = "yelp_subset_distilbert_regression_mask.csv"
text_name = "Word"

print(calculatePOSStatistics(method_path_name, lexica_path, curr_lex, text_name))

100%|██████████| 23283/23283 [00:00<00:00, 643939.36it/s]

{'PRON': 54, 'AUX': 19, 'ADV': 722, 'VERB': 3756, 'PART': 6, 'ADP': 76, 'DET': 38, 'NOUN': 8176, 'PUNCT': 88, 'SPACE': 0, 'ADJ': 2672, 'CCONJ': 10, 'SCONJ': 20, 'PROPN': 1402, 'NUM': 792, 'INTJ': 114, 'X': 326, 'SYM': 5, 'UNCLSFD': 5007, 'valid_count': 0}
